In [1]:
import pandas as pd
from kor.extraction import create_extraction_chain
from kor.nodes import Object, Text, Number
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import LlamaCpp
from tqdm import tqdm

In [2]:
df = pd.read_csv('./data_merge.csv')

FileNotFoundError: [Errno 2] No such file or directory: './data_merge.csv'

In [ ]:
print(df['meta'][0])


Дата выхода: 2023-08-02
В 2023 году в Китае было приобретено слияние оборудования для смены коробок (первый пункт: винты)
(Контингент: CGN-202305310008)
Совместимая закупка оборудования для смены коробок в 2023 году (первый пункт: Хиннин) (номер тендера: CGN-
202305310008), идентифицированные лица:
1. Информация о кандидатах:
Владелец: SMS
Цены: CNY 3 591 735.00
II. ДРУГИЕ ОБЯЗАТЕЛЬСТВА:
Нет
III. УПРАВЛЕНИЕ ВЕРХОВНОГО КОМИССАРА
Надзор за осуществлением данного тендерного проекта /.
IV. Связи
Участники торгов: &lt; &lt; Гуаньчжоу &gt; &gt;
Свяжитель: Белый
Тел.: 010-63711985
Электронная почта: baishaopeng@cgnpc.com.cn
Агент по торгам: &quot; Чайна инжиниринг лимитед &quot;
Контактное лицо: Пенсильван
Тел.: 0755-8447860
Электронная почта:
pengxiongbing@cgnpc.com.cn
Участник торгов или его главный руководитель (руководитель проекта): (подпись)
Участник торгов или его агент: (глава)





In [3]:
df['price_raw'] = [x.split('II.')[0] for x in df['meta']]
df['price_raw'] = [x.split('Цен')[-1] for x in df['price_raw']]

NameError: name 'df' is not defined

In [4]:
schema = Object(
    id="tender_info",
    description="You need to find the number representing the amount in the provided text.",
    attributes=[
        Number(
            id="price",
            description="bid amount",
            examples=[("а: RMB: 730,000.00\n", 730000), ('а: RMB7 607 834.00\n', 607834), ('ы: ¥2 526 800,00\n', 2526800), ('nСредняя цена: 5 685 473,00\n', 5685473), 
                      ('CNY 36, 599, 390.00', 36599390)],
        ),
    ],
    examples=[
        (
            """
            ы: ¥48 021 826.00\n
            """,
            [
                {"price": 48021826}
            ],
        ),
        (
            """
            Дата выхода: 2023-07-19\nИнструкции результатов в рамках проекта поддержки разработки программного обеспечения компании IQ (показатель 02 - умный бизнес)\n(Контингент: CGN-202303030004)\nПроект поддержки разработки программного обеспечения для компании IQ (02_ Интеллектуальный синергетический бизнес) (номер тендера: CGN-\n202303030004) В рамках проекта были определены следующие участники:\n1. Информация о кандидатах:\nПроект: Программа поддержки разработки программного обеспечения компании IQ (показатель 02 - Интеллектуальный бизнес)\nПроизводитель: China Tech\nСредняя цена: 96450000.00\n
            """,
            [
                {"price": 96450000}
            ], 
        ),
    ],
    many=True,
)

In [16]:
llm = LlamaCpp(
    model_path="../models/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf",
    n_gpu_layers=50,
    n_batch=512,
    n_ctx=1024,
    f16_kv=True,
    callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]),
    verbose=True,
    temperature = 0
)

llama_model_loader: loaded meta data with 24 key-value pairs and 291 tensors from ../models/Mistral-7B-Instruct-v0.2-GGUF/mistral-7b-instruct-v0.2.Q8_0.gguf (version GGUF V3 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q8_0     [  4096, 32000,     1,     1 ]
llama_model_loader: - tensor    1:              blk.0.attn_q.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    2:              blk.0.attn_k.weight q8_0     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    3:              blk.0.attn_v.weight q8_0     [  4096,  1024,     1,     1 ]
llama_model_loader: - tensor    4:         blk.0.attn_output.weight q8_0     [  4096,  4096,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_gate.weight q8_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.ffn_up.weight q8_0     [  4096, 14336,     1,     1 ]
llama_model_loader: - tensor    7:            blk.0.ffn_down

In [7]:
chain = create_extraction_chain(llm, schema, encoder_or_encoder_class='json')

In [ ]:
result = []
for i_text in tqdm(df['price_raw']):
    ans = chain.run(text=(i_text))
    result.append(ans['data'])

In [9]:
result

[{'tender_info': [{'price': 3591735}]},
 {'tender_info': [{'price': 64806000}]},
 {'tender_info': [{'price': 666628967}]},
 {'tender_info': [{'price': 62078826}]},
 {'tender_info': [{'price': 1242000000}]},
 {'tender_info': [{'price': 730000}]},
 {'tender_info': [{'price': 3680000}]},
 {'tender_info': [{'price': 15815020}]},
 {'tender_info': [{'price': 7607834}]},
 {'tender_info': [{'price': 18481366}]},
 {'tender_info': [{'price': 234112400}]},
 {'tender_info': [{'price': 20880000}]},
 {'tender_info': [{'price': 2257200}]},
 {'tender_info': [{'price': 8546000}]},
 {'tender_info': [{'price': 30750000}]},
 {'tender_info': [{'price': 188250000}]},
 {'tender_info': [{'price': 143000000}]},
 {'tender_info': [{'price': 2789120}]},
 {'tender_info': [{'price': 2350000}]},
 {'tender_info': [{'price': 13330000}]},
 {'tender_info': [{'price': 36599390}]},
 {'tender_info': [{'price': 1230000000}]},
 {'tender_info': [{'price': 3080000}]},
 {'tender_info': [{'price': 62880000}]},
 {'tender_info': [

In [42]:
temp_list = []
for i in tqdm(range(len(result))):
    if len(result[i]) > 0:
        temp_list.append(result[i]['tender_info'][0])
    else:
        temp_list.append({"price": ''})

100%|██████████| 608/608 [00:00<00:00, 1433466.46it/s]


In [44]:
temp_df = pd.DataFrame(temp_list)

In [45]:
temp_df.to_csv('./data/clear_price.csv', index=False)

In [46]:
temp_df

,price
0,3591735
1,64806000
2,666628967
3,62078826
4,1242000000
...,...
603,85935850
604,2599000
605,17165200
606,5211618.61
